**Demo code for Exploratory Report** <br>
https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca

In [1]:
#!pip install web3

In [2]:
import os
import logging
from chainlink_pricefeeds import (abi, config, pricefeed_contracts, get_event_abi, get_logs) # see chainlink_pricefeeds.py
import pandas as pd
from web3 import Web3
from eth_utils import (encode_hex, event_abi_to_log_topic)

######### Enable logging for debug
logging.basicConfig(
        format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
        level='INFO',
        datefmt='%Y-%m-%d %H:%M:%S',
        )


# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

w3_connection = {'ethereum' : Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))}
logging.info (f'Ethereum connected: {w3_connection["ethereum"].isConnected()}')


2022-05-15 18:09:36 <module> INFO Ethereum connected: True


In [3]:
from_block = 0

logging.info(f'from_block={from_block}')

df_columns = ['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'threshold', 'base', 'heartbeat', 'decimals', 'blockNumber','tx_hash']
df = pd.DataFrame(columns = df_columns)
    
event_AnswerUpdated = get_event_abi(abi, 'AnswerUpdated')
topic_AnswerUpdated  = encode_hex(event_abi_to_log_topic(event_AnswerUpdated))

for chain in config.keys():
    chain_contracts = config[chain]
    for ticker in chain_contracts.keys():
        w3 = w3_connection[chain_contracts[ticker][0]]
        address = chain_contracts[ticker][1]
        threshold = chain_contracts[ticker][2]
        base = chain_contracts[ticker][3]
        heartbeat = chain_contracts[ticker][4]
        
        decimals = w3.eth.contract(address=address, abi=abi).functions.decimals().call()

        df_ticker = get_logs(w3, address, [topic_AnswerUpdated], [event_AnswerUpdated], from_block)
        if len(df_ticker)>0:
            df_ticker[['chain', 'ticker', 'threshold', 'base', 'heartbeat', 'decimals', 'type']] = [chain , ticker, threshold, base, heartbeat, decimals, 'direct']

            df_ticker['tx_hash']       = df_ticker['transactionHash'].apply(lambda x: x.hex())        
            df_ticker['updated_at']    = df_ticker['args'].apply(lambda x: pd.to_datetime(x['updatedAt'], unit='s'))
            df_ticker['price']         = df_ticker['args'].apply(lambda x: x['current'])
            df_ticker['price_decimal'] = df_ticker['price']/10**(decimals)

            df = df.append(df_ticker[df_columns], ignore_index=True)
            logging.info(f'''{ticker.upper()}: observation count = {len(df_ticker)}, price range: {df_ticker['price_decimal'].min()}-{df_ticker['price_decimal'].max()}''')

            if ticker == 'eth-usd':
                df_eth_usd = df_ticker
            elif base == 'usd': # usd2eth cross rate
                ticker = ticker.replace('-usd','-eth')
                if ticker not in chain_contracts.keys():
                    df_cross = pd.DataFrame(columns = df_columns)
                    for k, t in df_ticker.iterrows(): 
                        e = df_eth_usd[df_eth_usd['updated_at'] <= t['updated_at']]
                        if len(e) > 0:
                            i = (t['updated_at'] - e['updated_at']).idxmin() 
                            e_price_decimal, e_decimals = e.loc[i][['price_decimal','decimals']]
                            t.ticker = ticker                       
                            t.price_decimal = t.price_decimal/e_price_decimal
                            t.decimals = 18
                            t.base = 'eth'                         
                            t.type = 'cross'
                            t.price = int(t.price_decimal*10**(t.decimals))
                            df_cross = df_cross.append(t)
                    df = df.append(df_cross, ignore_index=True)
                    logging.info(f'''cross rate {ticker.upper()}: observation count = {len(df_cross)}, price range: {df_cross['price_decimal'].min()}-{df_cross['price_decimal'].max()}''')
            elif base == 'eth': #eth2usd cross rate
                ticker = ticker.replace('-eth','-usd')
                if ticker not in chain_contracts.keys():
                    df_cross = pd.DataFrame(columns = df_columns)
                    for k, t in df_ticker.iterrows(): 
                        e = df_eth_usd[df_eth_usd['updated_at'] <= t['updated_at']]
                        if len(e) > 0:
                            i = (t['updated_at'] - e['updated_at']).idxmin() 
                            e_price_decimal, e_decimals = e.loc[i][['price_decimal','decimals']]
                            t.ticker = ticker
                            t.decimals = e_decimals
                            t.base = 'usd'                        
                            t.type = 'cross'
                            t.price_decimal = t.price_decimal*e_price_decimal
                            t.price = int(t.price_decimal*10**(t.decimals))
                            df_cross = df_cross.append(t)
                    df = df.append(df_cross, ignore_index=True)
                    logging.info(f'''cross rate {ticker.upper()}: observation count = {len(df_cross)}, price range: {df_cross['price_decimal'].min()}-{df_cross['price_decimal'].max()}''')
                
df = df.sort_values(by = 'updated_at').reset_index(drop=True)
pd.set_option("precision", 18)

2022-05-15 18:09:36 <module> INFO from_block=0
2022-05-15 18:09:40 get_logs INFO from_block 12000000 to_block 12300000
2022-05-15 18:09:40 get_logs INFO from_block 12300001 to_block 12600001
2022-05-15 18:09:46 get_logs INFO Total logs count 4220
2022-05-15 18:09:46 get_logs INFO from_block 12600002 to_block 12900002
2022-05-15 18:09:49 get_logs INFO Total logs count 7404
2022-05-15 18:09:49 get_logs INFO from_block 12900003 to_block 13200003
2022-05-15 18:09:51 get_logs INFO Total logs count 10147
2022-05-15 18:09:51 get_logs INFO from_block 13200004 to_block 13500004
2022-05-15 18:09:53 get_logs INFO Total logs count 13011
2022-05-15 18:09:53 get_logs INFO from_block 13500005 to_block 13800005
2022-05-15 18:09:56 get_logs INFO Total logs count 15747
2022-05-15 18:09:56 get_logs INFO from_block 13800006 to_block 14100006
2022-05-15 18:09:59 get_logs INFO Total logs count 18435
2022-05-15 18:09:59 get_logs INFO from_block 14100007 to_block 14400007
2022-05-15 18:10:01 get_logs INFO Tot

2022-05-15 18:12:47 get_logs INFO from_block 12300001 to_block 12600001
2022-05-15 18:12:49 get_logs INFO Total logs count 1557
2022-05-15 18:12:49 get_logs INFO from_block 12600002 to_block 12900002
2022-05-15 18:12:50 get_logs INFO Total logs count 3258
2022-05-15 18:12:50 get_logs INFO from_block 12900003 to_block 13200003
2022-05-15 18:12:51 get_logs INFO Total logs count 4938
2022-05-15 18:12:51 get_logs INFO from_block 13200004 to_block 13500004
2022-05-15 18:12:53 get_logs INFO Total logs count 6515
2022-05-15 18:12:53 get_logs INFO from_block 13500005 to_block 13800005
2022-05-15 18:12:54 get_logs INFO Total logs count 8074
2022-05-15 18:12:54 get_logs INFO from_block 13800006 to_block 14100006
2022-05-15 18:12:56 get_logs INFO Total logs count 9804
2022-05-15 18:12:56 get_logs INFO from_block 14100007 to_block 14400007
2022-05-15 18:12:58 get_logs INFO Total logs count 11259
2022-05-15 18:12:58 get_logs INFO from_block 14400008 to_block 14700008
2022-05-15 18:12:59 get_logs IN

2022-05-15 18:15:36 get_logs INFO Total logs count 8686
2022-05-15 18:15:36 get_logs INFO from_block 13500005 to_block 13800005
2022-05-15 18:15:38 get_logs INFO Total logs count 10620
2022-05-15 18:15:38 get_logs INFO from_block 13800006 to_block 14100006
2022-05-15 18:15:40 get_logs INFO Total logs count 12318
2022-05-15 18:15:40 get_logs INFO from_block 14100007 to_block 14400007
2022-05-15 18:15:41 get_logs INFO Total logs count 13226
2022-05-15 18:15:41 get_logs INFO from_block 14400008 to_block 14700008
2022-05-15 18:15:41 get_logs INFO Total logs count 13922
2022-05-15 18:15:41 get_logs INFO from_block 14700009 to_block 15000009
2022-05-15 18:15:42 get_logs INFO Total logs count 14541
2022-05-15 18:15:50 <module> INFO CRV-USD: observation count = 14541, price range: 1.06707709-6.72852622
2022-05-15 18:15:52 <module> INFO CVX-USD: observation count = 1024, price range: 10.02837476-47.16700316
2022-05-15 18:16:04 <module> INFO cross rate CVX-ETH: observation count = 1024, price ra

In [4]:
display(df)

,ticker,updated_at,chain,price,price_decimal,type,threshold,base,heartbeat,decimals,blockNumber,tx_hash,address,args,blockHash,event,logIndex,transactionHash,transactionIndex
0,comp-eth,2021-02-18 09:09:54,ethereum,244378668045116040,0.244378668045116043,direct,0.020000000000000000,eth,86400,18,11879906,0x9610367f2cd00cba48361195cb211f54e5775cf54efc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,crv-eth,2021-02-18 10:43:05,ethereum,1486000000000000,0.001486,direct,0.020000000000000000,eth,86400,18,11880352,0xa102d125a730c7186a458749e3a8de6323713567c6fa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,snx-eth,2021-02-18 10:55:03,ethereum,12479365000000000,0.012479364999999999,direct,0.020000000000000000,eth,86400,18,11880406,0x7537d4fce2d92483de4fd133947ec2cde6883f839685...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,crv-eth,2021-02-18 11:47:08,ethereum,1454500000000000,0.0014545,direct,0.020000000000000000,eth,86400,18,11880656,0xb170655b26a54ea69deb98102105e6b11a686338dcb1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ftm-eth,2021-02-18 12:23:53,ethereum,108390000000000,0.00010839,direct,0.029999999999999999,eth,86400,18,11880835,0x3a973394035ccfd333f19fbeb09003373e22a269331e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290711,fei-usd,2022-05-15 06:03:01,ethereum,100328742,1.00328741999999993,direct,0.010000000000000000,usd,3600,8,14778284,0xf2b4a456b53041eab1a1d0eede1956ce1d8e747f87ef...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290712,aave-usd,2022-05-15 06:04:06,ethereum,8712093383,87.120933829999998466,direct,0.010000000000000000,usd,3600,8,14778286,0x76c8bd4b5aa0a64c6d4792cc9dee91748a407e1cee04...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290713,spell-usd,2022-05-15 06:04:33,ethereum,131607,0.00131607,direct,0.010000000000000000,usd,86400,8,14778287,0x8644ccb9358c1147e0da4c412d1dfe0f7c855cd07ed6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290714,spell-eth,2022-05-15 06:04:33,ethereum,648003005486,0.000000648003005487,cross,0.010000000000000000,eth,86400,18,14778287,0x8644ccb9358c1147e0da4c412d1dfe0f7c855cd07ed6...,0x8640b23468815902e011948F3aB173E1E83f9879,"(current, roundId, updatedAt)",b'~\x88@DI\xb8\xa2_\xf8\xf4\xa2$d7e\x84\x00MI\...,AnswerUpdated,59.0,b'\x86D\xcc\xb95\x8c\x11G\xe0\xdaLA-\x1d\xfe\x...,32.0


# Exploratory report:

https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca